# RCS Benchmark

In [1]:
# Lowest qubit width
low = 8
# Highest qubit width
high = 30
# Samples to average per qubit width
samples = 1

In [2]:
import math
import os
import random
import time

from pyqrack import QrackSimulator

# This is the maximum qubit count you want to fit in a GPU "maximum allocation segment."
# (Your GPU probably has 4 such segments. You might want this 1 less than theoretical max,
# so that "memory fragmentation" doesn't prevent using more than 1 segment in total.)
os.environ['QRACK_MAX_PAGE_QB']='27'
# This is an overall allocation limit for your GPU(s), in megabytes.
# If you have multiple GPUs, you can list separate limits in device ID order,
# separated by a comma.
os.environ['QRACK_MAX_ALLOC_MB']='15872'
# This is the maximum total number of fully-entangled qubits you expect to achieve using QPager.
os.environ['QRACK_MAX_PAGING_QB']='30'
# This is the maximum total number of fully-entangled qubits you expect to fit in general RAM.
os.environ['QRACK_MAX_CPU_QB']='32'
# Above this threshold, "QTensorNetwork" restricts simulations to "past light cone."
# At or below the threshold, much more work can be reused.
os.environ['QRACK_QTENSORNETWORK_THRESHOLD_QB']='30'

## PyQrack

In [3]:
def bench_qrack(n):
    # This is basically a "quantum volume" (random) circuit.
    start = time.perf_counter()

    sim = QrackSimulator(n)

    lcv_range = range(n)
    all_bits = list(lcv_range)

    single_count = 0
    double_count = 0
    for _ in lcv_range:
        # Single-qubit gates
        for i in lcv_range:
            sim.u(i, random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi))

        # 2-qubit couplers
        unused_bits = all_bits.copy()
        random.shuffle(unused_bits)
        while len(unused_bits) > 1:
            sim.mcx([unused_bits.pop()], unused_bits.pop())

    # Terminal measurement
    # print(sim.m_all())
    sim.m_all()

    return time.perf_counter() - start

# Make sure the OpenCL environment is initialized before timing.
# (You probably also want to precompile OpenCL kernels with the `qrack_cl_compile` utility, in general.)
bench_qrack(1)

qrack_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_qrack(n))

    qrack_results[n] = sum(width_results) / samples
    print(n, ": ", qrack_results[n], " seconds")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
8 :  0.0023531320002803113  seconds
9 :  0.0032005989996832795  seconds
10 :  0.005233775000306196  seconds
11 :  0.0112397550001333  seconds
12 :  0.01594833600029233  seconds
13 :  0.016706120000890223  seconds
14 :  0.020815935000428  seconds
15 :  0.023386397999274777  seconds
16 :  0.027435580001110793  seconds
17 :  0.0302177980011038  seconds
18 :  0.038212395998925786  seconds
19 :  0.05120375900150975  seconds
20 :  0.11235168899838754  seconds
21 :  0.18833843799984606  seconds
22 :  0.40136326200081385  seconds
23 :  0.7781868559995928  seconds
24 :  1.7914650590009842  seconds
25 :  3.6986852419995557  seconds
26 :  7.785492257000442  seconds
27 :  16.371238868001456  seconds
28 :  38.1144417029991  seconds
29 :  79.25250359200072  seconds
30 :  183.64159018899954  seconds
